In [1]:
import sys
import os
import shutil
import subprocess
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QTextEdit, QVBoxLayout, QHBoxLayout, QWidget, QFrame
from PyQt5.QtCore import Qt, pyqtSignal, QTimer, QThread, pyqtSlot
from PyQt5.QtGui import QPixmap, QFont, QColor

class DropLabel(QLabel):
    dropSignal = pyqtSignal(str)

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.setAcceptDrops(True)

    def dragEnterEvent(self, event):
        if event.mimeData().hasUrls():
            event.accept()
        else:
            event.ignore()

    def dropEvent(self, event):
        for url in event.mimeData().urls():
            file_path = url.toLocalFile()
            self.dropSignal.emit(file_path)

class ProcessThread(QThread):
    finished = pyqtSignal(str)
    error = pyqtSignal(str)

    def __init__(self, script_path):
        QThread.__init__(self)
        self.script_path = script_path

    def run(self):
        try:
            result = subprocess.run(['python3', self.script_path], 
                                    capture_output=True, text=True, check=True, timeout=60)
            self.finished.emit(result.stdout)
        except subprocess.CalledProcessError as e:
            self.error.emit(f"Error: {e.stderr}")
        except subprocess.TimeoutExpired:
            self.error.emit("Error: Script execution timed out")
        except Exception as e:
            self.error.emit(f"Unexpected error: {str(e)}")

class ImageFrame(QFrame):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.setObjectName("imageFrame")
        self.setStyleSheet("""
            QFrame#imageFrame {
                background-color: white;
                border-radius: 10px;
                border: 1px solid #e0e0e0;
            }
        """)

class ImageProcessorGUI(QMainWindow):
    def __init__(self):
        super().__init__()
        # Set up paths
        current_dir = os.getcwd()  # Get current working directory
        project_root = os.path.dirname(current_dir)  # Assuming we're in the 'Scripts' folder
        images_dir = os.path.join(project_root, 'Images', 'GUI')
        self.raw_images_folder = os.path.join(images_dir, 'Raw Images GUI')
        self.output_folder = os.path.join(images_dir, 'Processed Images GUI')
        self.inference_script = os.path.join(current_dir, 'inference_numbering_gui.py')
        
        self.initUI()

    def initUI(self):
        self.setWindowTitle('MTD Classifier')
        self.setGeometry(100, 100, 1000, 700)
        self.setStyleSheet("""
            QMainWindow {
                background-color: #f5f5f5;
            }
            QLabel {
                color: #333333;
                font-family: 'Segoe UI', Arial, sans-serif;
            }
            QTextEdit {
                font-family: 'Consolas', 'Courier New', monospace;
                font-size: 12px;
                background-color: #2b2b2b;
                color: #f0f0f0;
                border-radius: 5px;
            }
        """)

        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        layout = QVBoxLayout(central_widget)
        layout.setSpacing(10)
        layout.setContentsMargins(20, 20, 20, 20)

        image_layout = QHBoxLayout()

        self.rawImageFrame = ImageFrame()
        raw_layout = QVBoxLayout(self.rawImageFrame)
        self.rawImageLabel = DropLabel('Drop Raw Image Here')
        self.rawImageLabel.setAlignment(Qt.AlignCenter)
        self.rawImageLabel.setStyleSheet('font-size: 18px; color: #757575;')
        self.rawImageLabel.setMinimumSize(350, 350)
        self.rawImageLabel.dropSignal.connect(self.handleDroppedImage)
        raw_layout.addWidget(self.rawImageLabel)

        self.processedImageFrame = ImageFrame()
        processed_layout = QVBoxLayout(self.processedImageFrame)
        self.processedImageLabel = QLabel('Processed Image')
        self.processedImageLabel.setAlignment(Qt.AlignCenter)
        self.processedImageLabel.setStyleSheet('font-size: 18px; color: #757575;')
        self.processedImageLabel.setMinimumSize(350, 350)
        processed_layout.addWidget(self.processedImageLabel)

        image_layout.addWidget(self.rawImageFrame)
        image_layout.addWidget(self.processedImageFrame)

        self.logTextEdit = QTextEdit()
        self.logTextEdit.setReadOnly(True)
        self.logTextEdit.setMinimumHeight(100)

        layout.addLayout(image_layout, 3)
        layout.addWidget(self.logTextEdit, 1)

    def clearFolders(self):
        for folder in [self.raw_images_folder, self.output_folder]:
            if os.path.exists(folder):
                for filename in os.listdir(folder):
                    file_path = os.path.join(folder, filename)
                    try:
                        if os.path.isfile(file_path) or os.path.islink(file_path):
                            os.unlink(file_path)
                        elif os.path.isdir(file_path):
                            shutil.rmtree(file_path)
                    except Exception as e:
                        self.logTextEdit.append(f'Failed to delete {file_path}. Reason: {e}')
            else:
                os.makedirs(folder, exist_ok=True)
        
        self.processedImageLabel.clear()
        self.processedImageLabel.setText('Processed Image')

    def handleDroppedImage(self, file_path):
        self.clearFolders()
        file_name = os.path.basename(file_path)
        destination = os.path.join(self.raw_images_folder, file_name)
        os.makedirs(os.path.dirname(destination), exist_ok=True)
        shutil.copy2(file_path, destination)
        self.logTextEdit.append(f'Processing {file_name}...')
        self.updateRawImagePreview(destination)
        
        QTimer.singleShot(100, lambda: self.processImage(destination))

    def updateRawImagePreview(self, file_path):
        pixmap = QPixmap(file_path)
        scaled_pixmap = pixmap.scaled(self.rawImageLabel.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation)
        self.rawImageLabel.setPixmap(scaled_pixmap)
        self.rawImageLabel.setText('')

    def processImage(self, image_path):
        self.logTextEdit.append('Processing image...')
        QApplication.processEvents()

        if not os.path.exists(self.inference_script):
            self.logTextEdit.append(f"Error: Cannot find {self.inference_script}")
            return

        self.process_thread = ProcessThread(self.inference_script)
        self.process_thread.finished.connect(self.onProcessingFinished)
        self.process_thread.error.connect(self.onProcessingError)
        self.process_thread.start()

    @pyqtSlot(str)
    def onProcessingFinished(self, output):
        self.logTextEdit.append('Processing complete')
        QTimer.singleShot(1000, self.updateProcessedImagePreview)  # Wait 1 second before updating

    @pyqtSlot(str)
    def onProcessingError(self, error_message):
        self.logTextEdit.append(f"Processing error: {error_message}")

    def updateProcessedImagePreview(self):
        try:
            files = [f for f in os.listdir(self.output_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
            if not files:
                return
            
            file_path = os.path.join(self.output_folder, files[0])
            
            pixmap = QPixmap(file_path)
            scaled_pixmap = pixmap.scaled(self.processedImageLabel.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation)
            self.processedImageLabel.setPixmap(scaled_pixmap)
            self.processedImageLabel.setText('')
            
            filename = os.path.basename(file_path)
            usability = 'Usable' if filename.startswith('numbered_') else 'Unusable'
            self.logTextEdit.append(f'Result: {usability}')
        except Exception as e:
            self.logTextEdit.append(f'Error updating processed image preview: {str(e)}')

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = ImageProcessorGUI()
    ex.show()
    sys.exit(app.exec_())

QSocketNotifier: Can only be used with threads started with QThread


SystemExit: 0

/home/hogglet/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
